<a href="https://colab.research.google.com/github/rajithasuja/Capstone_Project_2/blob/main/The_Battle_of_Neighborhoods_week_1_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project - The Battle of Neighborhood (Week 1)

In [1]:
# importing all required libraries

import numpy as np
import pandas as pd
import time
import requests
import folium
import json
from geopy.geocoders import Nominatim

## **1. Introduction Section**

### **Description of the Problem and Background**

#### **Scenario:**
I am a Data Scientist residing in India. I currently live within walkable distance to grocery store, reputed educational institutes, malls and entertainment zones. I have received an offer letter for MS in University of Washington, USA which offers me a good scholarship. The key question is: How can I find a similar location to mine at Wahington. Certainly, I could use available realestate apps and google but the idea is to make a comparision and evaluation of the rental options at Seattle, Washington.The apartment at Seattle must meet the following demands:



*   apartment must be 1 or 2 BHK
*   Desired location must be close to my university
* price shouldn't exceed $6,000
* desirable to have venues such as grocery store, coffee shops, Asian restaurants and food shops.

##### **Business Problem:**
The challenge is to find a suitable house that meets the above mentioned demands.

##### **Interested Audience:**
I believe this is a relevant challenge with valid questions. This is very helpful, especially for students who apply abroad or masters and for better jobs.


# **2. Data Section**

#### **Description of the data and its sources that will be used to solve the problem**
##### **Description of the data:**
The following data is requires to answer the issues of the problem:

* List of Boroughs and Neigborhoods in Seattle along with their latitude and longitude
* List of apartments close to the college in the area which meets my demands.
* Venues for shopping malls and food joints.

#### **How the data will be use to solve the problem**
The data will be used as follows:
* Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)
* Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately
*Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
* Create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
* Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
* Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.


The procesing of these DATA will allow to answer the key questions to make a decision:

* What is the cost of rent (per square ft) around a mile radius from the university?
*What is the area of Seattle with best rental pricing that meets criteria established?
*What is the distance from university and the tentative future home?
*What are the venues of the two best places to live? How the prices compare?
*How venues distribute among Seattle neighborhoods and around metro stations?
*Are there tradeoffs between size and price and location?
*Any other interesting statistical data findings of the real estate and overall data.

In [5]:
address = 'Gachibowli, India'
geolocator = Nominatim()
location = geolocator.geocode(address)
lat = location.latitude
long = location.longitude
print('The geograpical coordinate of Indian home are {}, {}.'.format(lat, long))

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Indian home are 17.4436222, 78.3519638.


In [6]:
neighborhood_latitude = 17.4436222
neighborhood_longitude = 78.3519638

In [15]:
CLIENT_ID = 'UBWKQTDZTXRZWTCG01V4AWC5AAQNUIAXBXREJY1LNWPXPRKD'
CLIENT_SECRET = 'ZEQP4WMJUFE05PNGBR5JHOPICN4Z2DPEMJPNFGCJZ0HR2TXH'
VERSION = '20210105'

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=UBWKQTDZTXRZWTCG01V4AWC5AAQNUIAXBXREJY1LNWPXPRKD&client_secret=ZEQP4WMJUFE05PNGBR5JHOPICN4Z2DPEMJPNFGCJZ0HR2TXH&v=20210105&ll=17.4436222,78.3519638&radius=500&limit=100'

In [24]:
results = requests.get(url).json()

In [25]:
def get_category_type(row):
  try:
    categories_list = row['categories']
  except:
    categories_list = row['venue.categories']
  if len(categories_list)==0:
    return None
  else:
    return categories_list[0]['name']

In [26]:
results

{'meta': {'code': 200, 'requestId': '5ff454baf6bc1c09d3ced1b5'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-55733ab8498e4ce267bf22ab-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1c9941735',
         'name': 'Ice Cream Shop',
         'pluralName': 'Ice Cream Shops',
         'primary': True,
         'shortName': 'Ice Cream'}],
       'id': '55733ab8498e4ce267bf22ab',
       'location': {'cc': 'IN',
        'country': 'India',
        'distance': 379,
        'formattedAddress': ['India'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 17.44299847823975,
          'lng': 78.35547531227311}],
        'lat': 17.44299847823975,
        'lng': 78.35547531227311},
       'na

In [29]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
INnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
INnearby_venues =INnearby_venues.loc[:, filtered_columns]

# filter the category for each row
INnearby_venues['venue.categories'] = INnearby_venues.apply(get_category_type, axis=1)

# clean columns
INnearby_venues.columns = [col.split(".")[-1] for col in INnearby_venues.columns]

INnearby_venues.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,creamstone,Ice Cream Shop,17.442998,78.355475
1,Karachi Bakery,Bakery,17.442930,78.355336
2,Chettinaduvilas,Food Truck,17.442858,78.356053
3,Gachibowli Stadium,Stadium,17.445982,78.348245
4,Cafe Coffee Day,Café,17.442771,78.355737
5,Mustang terrace,Italian Restaurant,17.442840,78.355320
6,Adithya inn,Hotel,17.443037,78.354475
7,Cibo House,Italian Restaurant,17.443122,78.354781
8,Ratnadeep Super Market,Grocery Store,17.442863,78.355080
9,Star Hypermarket,Department Store,17.444351,78.355490


In [31]:
# create map of Singapore place  using latitude and longitude values
map_in = folium.Map(location=[lat, long], zoom_start=20)

# add markers to map
for lat, lng, label in zip(INnearby_venues['lat'], INnearby_venues['lng'], INnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_in)  
    
map_in